# Human Activity Recognition (HAR) with MotionSense Dataset (CWT + CNN Approach)

In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal
import pywt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import seaborn as sns

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [ ]:
# Sets the device on which PyTorch tensors and models will be allocated
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Prints the selected device to confirm where computations will run
print(f"Using device: {device}")

In [ ]:
# Imports the Colab-specific module to interact with Google Drive
from google.colab import drive
# Mounts the Google Drive to the Colab-environment
drive.mount('/content/drive')
# Defines a string variable DATA_PATH_PREFIX that stores the path to a specific folder in the Drive
# Base path to access data files
DATA_PATH_PREFIX = "/content/drive/MyDrive/EEE4114F - ML/"

In [16]:
def get_ds_infos():
    """
    Read the file includes data subject information.
    Returns:
        A pandas DataFrame that contains information about data subjects' attributes
    """
    # try block to handle errors
    try:
        # Attempts to load a CSV file named data_subjects_info.csv from the given DATA_PATH_PREFIX path
        # dss holds the loaded DataFrame
        dss = pd.read_csv(DATA_PATH_PREFIX + "data_subjects_info.csv")
        # Prints a success message if the file loads without issues
        print("[INFO] -- Data subjects' information is imported.")
    # Catches the error if the file is not found at the specific path
    except FileNotFoundError:
        # Prints an error message
        print(f"[ERROR] -- data_subjects_info.csv not found at {DATA_PATH_PREFIX + 'data_subjects_info.csv'}")
        print("Please ensure DATA_PATH_PREFIX is set correctly and the file exists.")
        # If the file is not found, it returns an empty DataFrame with a single column named 'code'
        return pd.DataFrame(columns=['code'])
    # if the file was successfully read, the loaded DataFrame (stored in dss) is returned
    return dss

In [17]:
def set_data_types(data_types=["userAcceleration"]):
    """
    Select the sensors and return a list of lists of column names.
    Args:
        data_types: A list of sensor data type from this list: [attitude, gravity, rotationRate, userAcceleration]
    Returns:
        A list of lists, where each inner list contains the column names for a sensor type (e.g., [t+".x",t+".y",t+".z"]).
    """
    # Iintializes an empty list to store the results
    dt_list_groups = []
    # Loop through each sensor type specified in the data_types list
    for t in data_types:
        # Check if the sensor type is not attitude
        # Yes: Construst a list of the 3-axis column names and append it to the result list
        if t != "attitude":
            dt_list_groups.append([t+".x", t+".y", t+".z"])
        else:
            # Attitude uses orientation angle
            dt_list_groups.append([t+".roll", t+".pitch", t+".yaw"])
    # Returns the list of column names
    return dt_list_groups

In [18]:
def create_time_series(dt_list_groups_config, act_labels_config, trial_codes_config_map, mode="raw", labeled=True):
    """
    Creates a time-series DataFrame from raw sensor data files.
    Args:
        dt_list_groups_config: List of lists of feature column names, grouped by sensor type.
        act_labels_config: List of activity names.
        trial_codes_config_map: Dictionary mapping activity names to lists of trial codes.
        mode: "raw" (magnitude mode is not fully supported by this simplified version).
        labeled: Boolean, True if activity labels should be included.
    Returns:
        A tuple: (full_dataset_df, feature_cols_flat_list)
            full_dataset_df: Pandas DataFrame with all time-series data and labels.
            feature_cols_flat_list: Flat list of all feature column names.
    """
    # Flattens the list of lists into a single list of feature column names
    feature_cols_flat_list = [col for group in dt_list_groups_config for col in group]
    num_feature_cols = len(feature_cols_flat_list)

    # Initializes the column names for the final DataFrame
    column_names_for_df = feature_cols_flat_list[:]
    # Adds "act" column if activity labes are included
    if labeled:
        column_names_for_df.append("act")

    # Loads subject metada
    all_trial_dfs = []
    ds_list = get_ds_infos()
    # Check if subject info is missing
    # Yes: Prints an error message, aborts and returns an empty DataFrame
    if ds_list.empty and 'code' not in ds_list.columns:
        print("[ERROR] -- Cannot proceed without subject information.")
        return pd.DataFrame(columns=column_names_for_df), feature_cols_flat_list


    print("[INFO] -- Creating Time-Series")
    # Loops hrough all subjects and trials
    for sub_id in ds_list["code"]:
        for act_id, act_name in enumerate(act_labels_config):
            # Checks if no trials are configure
            # Yes:  Skips the activity
            if act_name not in trial_codes_config_map:
                print(f"[WARNING] -- No trial codes found for activity: {act_name}. Skipping.")
                continue
            for trial_code in trial_codes_config_map[act_name]:
                # Constructs the full file path for each subject's trial
                fname = f'{DATA_PATH_PREFIX}A_DeviceMotion_data/A_DeviceMotion_data/{act_name}_{trial_code}/sub_{int(sub_id)}.csv'
                # try block to handle errors
                try:
                    # Attampts to read the CSV file
                    raw_data_per_trial = pd.read_csv(fname)
                # Catches the error if the file is not found
                except FileNotFoundError:
                    # Prints a warninng message
                    print(f"[WARNING] -- File not found: {fname}. Skipping.")
                    continue

                # Drops unwanted index columns automatically added by pandas during saving
                raw_data_per_trial = raw_data_per_trial.drop(['Unnamed: 0'], axis=1, errors='ignore')

                # Extracts only the relevant feature columns for this trial
                current_trial_features = raw_data_per_trial[feature_cols_flat_list].values

               # Checks if labeled
               # Creates an array of labels with the same numbeer as the featires
                if labeled:
                    labels_for_this_trial = np.full((len(raw_data_per_trial), 1), act_id)
                    trial_data_np = np.concatenate((current_trial_features, labels_for_this_trial), axis=1)
                else:
                    trial_data_np = current_trial_features
                # Converts the current trial into a DataFrame and apends to the list of all trials
                all_trial_dfs.append(pd.DataFrame(data=trial_data_np, columns=column_names_for_df))

    # Checks if no trails loaded successfully
    # Yes: Returns an empty DataFrame
    if not all_trial_dfs:
        print("[ERROR] -- No data successfully loaded. Please check file paths, data structure, and configurations.")
        return pd.DataFrame(columns=column_names_for_df), feature_cols_flat_list

    print(f"[INFO] -- Concatenating {len(all_trial_dfs)} individual trial DataFrames.")
    # Concatenates all DataFrames into one final DataFrame
    full_dataset_df = pd.concat(all_trial_dfs, ignore_index=True)
    # Returns the complete time-series dataset (full_dataset_df) and a list of column names for model input (feature_cols_flat_list)
    return full_dataset_df, feature_cols_flat_list

In [19]:
def apply_cwt_to_signal(signal_data, scales=None, wavelet='morlet', sampling_rate=50):
    """
    Apply Continuous Wavelet Transform to a 1D signal.
    Args:
        signal_data: 1D numpy array representing the signal
        scales: Array of scales for CWT. If None, auto-generated
        wavelet: Type of wavelet to use (default: 'morlet')
        sampling_rate: Sampling rate of the signal in Hz
    Returns:
        cwt_coeffs: 2D array of CWT coefficients (scales x time)
        frequencies: Corresponding frequencies for each scale
    """
    if scales is None:
        frequencies = np.logspace(np.log10(1), np.log10(25), 30)
        scales = pywt.frequency2scale(wavelet, frequencies) * sampling_rate

    # Compute CWT coefficients
    cwt_coeffs, frequencies = pywt.cwt(signal_data, scales, wavelet, sampling_period=1/sampling_rate)

    # Extracts the magnitude of complex coefficients
    cwt_coeffs = np.abs(cwt_coeffs)
    # Returns:
    #   cwt_coeff: Magnitude of the wavelet coefficients.
    #   frequencies: Frequencies associated with each scale used
    return cwt_coeffs, frequencies

In [20]:
def create_cwt_windows_from_df(data_df, feature_cols, label_col, window_size, stride,
                              scales=None, wavelet='morlet', sampling_rate=50):
    """
    Creates windowed CWT data from a DataFrame.
    Args:
        data_df: Input DataFrame with features and labels.
        feature_cols: List of feature column names.
        label_col: Name of the label column.
        window_size: Size of each window.
        stride: Stride between windows.
        scales: CWT scales (if None, auto-generated)
        wavelet: Wavelet type for CWT
        sampling_rate: Sampling rate in Hz
    Returns:
        A tuple (np.array(X_cwt_windows), np.array(Y_labels)).
        X_cwt_windows shape: (num_windows, num_features, num_scales, window_size)
        Y_labels shape: (num_windows,)
    """
    X_cwt_windows_list = []
    Y_labels_list = []

    feature_data_np = data_df[feature_cols].values
    label_data_np = data_df[label_col].values

    print(f"[INFO] -- Applying CWT to {len(feature_cols)} features...")

    # Generate scales if not provided
    if scales is None:
        frequencies = np.logspace(np.log10(1), np.log10(25), 30)
        scales = pywt.frequency2scale(wavelet, frequencies) * sampling_rate

    num_scales = len(scales)
    print(f"[INFO] -- Using {num_scales} CWT scales")

    for i in range(0, len(feature_data_np) - window_size + 1, stride):
        window_features = feature_data_np[i : i + window_size]  # Shape: (window_size, num_features)
        window_labels_raw = label_data_np[i : i + window_size]

        # Apply CWT to each feature in the window
        cwt_features_list = []
        for feature_idx in range(len(feature_cols)):
            signal = window_features[:, feature_idx]
            cwt_coeffs, _ = apply_cwt_to_signal(signal, scales, wavelet, sampling_rate)
            cwt_features_list.append(cwt_coeffs)  # Shape: (num_scales, window_size)

        # Stack CWT features: (num_features, num_scales, window_size)
        cwt_window = np.stack(cwt_features_list, axis=0)
        X_cwt_windows_list.append(cwt_window)

        # Use mode of labels in the window as the window's label
        label_counts = np.bincount(window_labels_raw.astype(int))
        mode_label = np.argmax(label_counts)
        Y_labels_list.append(mode_label)

        if (len(X_cwt_windows_list) % 100) == 0:
            print(f"[INFO] -- Processed {len(X_cwt_windows_list)} windows...")

    return np.array(X_cwt_windows_list), np.array(Y_labels_list)

In [ ]:
# Defines a list of activity labels
ACT_LABELS = ["dws","ups", "wlk", "jog", "std", "sit"]
# Creates a mapping between each activity and its corresponding trial codes
TRIAL_CODES = {
    ACT_LABELS[0]:[1,2,11], #dws: trials 1, 2, 11
    ACT_LABELS[1]:[3,4,12], #ups: trials 3, 4, 12
    ACT_LABELS[2]:[7,8,15], #wlk: trials 7, 8, 15
    ACT_LABELS[3]:[9,16],   #jog: trials 9, 16
    ACT_LABELS[4]:[6,14],   #std: trials 6, 14
    ACT_LABELS[5]:[5,13]    #sit: trials 5, 13
}
# Specifies which sensor data types to include from each CSV file
# Available options: "attitude", "gravity", "rotationRate", "userAcceleration"
SELECTED_SENSOR_DATA_TYPES = ["userAcceleration", "rotationRate"]
# Prints the current configuration
print(f"[INFO] -- Selected sensor data types: {SELECTED_SENSOR_DATA_TYPES}")
print(f"[INFO] -- Selected activities: {ACT_LABELS}")

In [ ]:
# Calls the set_data_types() function with the selected sensor types (["userAcceleration", "rotationRate"])
# Returns a list of feature column names grouped by sensor type
dt_list_feature_groups = set_data_types(SELECTED_SENSOR_DATA_TYPES)
# Runs the data loading pipleline:
# - Loads CSVs for each subject/activity/trial
# - Extracts the relevant sensor feature
# - Attaches activity labels
# Returns:
# - dataset: contains all time-series data across subjects/trials
# - feature_columns_list: A list of selected feature column names
dataset, feature_columns_list = create_time_series(dt_list_feature_groups, ACT_LABELS, TRIAL_CODES, mode="raw", labeled=True)

# Checks if no data was successfully loaded
# Yes: Prints an error message and termitnaes executio
if dataset.empty:
    print("[STOP] -- Dataset is empty. Halting execution. Check file paths and data availability.")
    exit()
# Data Loaded Successfully
else:
    # Prints the shae of the DataFrame
    print(f"[INFO] -- Shape of raw time-Series dataset: {dataset.shape}")
    # Promts the first few rows of data using head()
    print(dataset.head())
    # Number of input features per time step
    NUM_FEATURES = len(feature_columns_list)
    # Number of unique activity labels
    NUM_CLASSES = len(ACT_LABELS)

In [23]:
# Number of samples in each sliding window segment.
WINDOW_SIZE = 128
# Step size for moving the window across the time series.
STRIDE = 64
SAMPLING_RATE = 50
CWT_SCALES = None
WAVELET_TYPE = 'morl'

# Sampling frequency of the sensor data in Hz (samples per second).
SAMPLING_RATE = 50

# Scales to be used in Continuous Wavelet Transform (CWT).
CWT_SCALES = None

# Type of wavelet to be used in CWT → 'morl': Morlet wavelet
WAVELET_TYPE = 'morl'


In [24]:
# Initialize a StandardScaler to normalize the feature data.
scaler = StandardScaler()
# Create a copy of the original dataset
scaled_dataset = dataset.copy()
# Apples the scaler only to the selected feature columns.
scaled_dataset[feature_columns_list] = scaler.fit_transform(dataset[feature_columns_list])
print("[INFO] -- Features scaled using StandardScaler.")
# Prints confirmation output that feature scaling is complete.
print("[INFO] -- Features scaled using StandardScaler.")


[INFO] -- Features scaled using StandardScaler.


In [ ]:
# Creates CWT windows from the scaled dataset
# Applies Continuous Wavelet Transform to each window of the time-series data
print("[INFO] -- Starting CWT transformation and windowing...")
X_cwt_windowed, Y_windowed = create_cwt_windows_from_df(
    scaled_dataset, feature_columns_list, 'act',
    WINDOW_SIZE, STRIDE, CWT_SCALES, WAVELET_TYPE, SAMPLING_RATE
)

# Prints shapes of resulting CWT feature array and corresponding labels
print(f"[INFO] -- Shape of CWT windowed X: {X_cwt_windowed.shape}")
print(f"[INFO] -- Shape of windowed Y: {Y_windowed.shape}")

# Extracts dimensions from the CWT-transformed windowed data
num_windows, num_features, num_scales, window_size = X_cwt_windowed.shape

# Reshape the 4D CWT array to 3D for compatibility with CNN input:
X_cwt_reshaped = X_cwt_windowed.reshape(num_windows, num_features * num_scales, window_size)


# Prints final shape of reshaped data
print(f"[INFO] -- Reshaped CWT data for CNN: {X_cwt_reshaped.shape}")

In [ ]:
# Split the CWT-transformed and reshaped dataset into 80% for training and 20% for test sets.
X_train, X_test, Y_train, Y_test = train_test_split(
    X_cwt_reshaped, Y_windowed, test_size=0.2, random_state=42, stratify=Y_windowed
)

# Prints the shapes pf the resulting train and test sets
print(f"X_train shape: {X_train.shape}, Y_train shape: {Y_train.shape}")
print(f"X_test shape: {X_test.shape}, Y_test shape: {Y_test.shape}")

In [ ]:
class MotionSensePyTorchDataset(Dataset):
    def __init__(self, X_data, Y_data):
        # Converts input data to float32 tensors
        self.X = torch.tensor(X_data, dtype=torch.float32)
        # Converts labels to long tensors (required by CrossEntropyLoss)
        self.Y = torch.tensor(Y_data, dtype=torch.long)

    def __getitem__(self, index):
        # Returns a single sample and its label at the given index
        return self.X[index], self.Y[index]

    def __len__(self):
        # Returns the total number of samples in the dataset
        return len(self.X)

# Instantiates PyTorch Datasets for training and testing sets
train_torch_dataset = MotionSensePyTorchDataset(X_train, Y_train)
test_torch_dataset = MotionSensePyTorchDataset(X_test, Y_test)

# Sets batch size for data loading
BATCH_SIZE = 64

# Creates DataLoaders to load data in batches
train_loader = DataLoader(train_torch_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_torch_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Prints confirmation output that PyTorch DataLoaders have been created
print("[INFO] -- PyTorch DataLoaders created.")

In [28]:
class CWT_CNN_HAR_Model(nn.Module):
    def __init__(self, num_input_channels, num_activity_classes, sequence_length):
        super(CWT_CNN_HAR_Model, self).__init__()

        # First convolutional layer
        self.conv_layer1 = nn.Conv1d(in_channels=num_input_channels, out_channels=128, kernel_size=9, padding='same')
        self.bn1 = nn.BatchNorm1d(128)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool1d(kernel_size=2, stride=2)
        self.dropout1 = nn.Dropout(0.3)

        # Second convoluional layer
        self.conv_layer2 = nn.Conv1d(in_channels=128, out_channels=256, kernel_size=7, padding='same')
        self.bn2 = nn.BatchNorm1d(256)
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool1d(kernel_size=2, stride=2)
        self.dropout2 = nn.Dropout(0.3)

        # Third convolutional layer
        self.conv_layer3 = nn.Conv1d(in_channels=256, out_channels=512, kernel_size=5, padding='same')
        self.bn3 = nn.BatchNorm1d(512)
        self.relu3 = nn.ReLU()
        self.maxpool3 = nn.MaxPool1d(kernel_size=2, stride=2)
        self.dropout3 = nn.Dropout(0.3)

        # Computes the size of the flattened feature vector after the convolutions
        flattened_output_size = 512 * (sequence_length // 8)

        # Fully connected layers for classification
        self.flatten_layer = nn.Flatten()
        self.fc_layer1 = nn.Linear(flattened_output_size, 256)
        self.relu4 = nn.ReLU()
        self.dropout4 = nn.Dropout(0.5)

        # Final output layer mapping to activity classes
        self.fc_layer2 = nn.Linear(256, 128)
        self.relu5 = nn.ReLU()
        self.dropout5 = nn.Dropout(0.5)

        self.fc_layer3 = nn.Linear(128, num_activity_classes)

    def forward(self, x_input):
         # Forward pass through the networ
        x = self.dropout1(self.maxpool1(self.relu1(self.bn1(self.conv_layer1(x_input)))))
        x = self.dropout2(self.maxpool2(self.relu2(self.bn2(self.conv_layer2(x)))))
        x = self.dropout3(self.maxpool3(self.relu3(self.bn3(self.conv_layer3(x)))))

        x = self.flatten_layer(x)
        x = self.dropout4(self.relu4(self.fc_layer1(x)))
        x = self.dropout5(self.relu5(self.fc_layer2(x)))
        output_logits = self.fc_layer3(x)
        return output_logits

In [ ]:
# Calculates the number of input channels after CWT transformation
NUM_CWT_CHANNELS = num_features * num_scales

# Instantiates the model and moves the model to approriate device
model = CWT_CNN_HAR_Model(
    num_input_channels=NUM_CWT_CHANNELS,
    num_activity_classes=NUM_CLASSES,
    sequence_length=WINDOW_SIZE
).to(device)

# Prints model summary and configuration info
print("[INFO] -- CWT-CNN Model instantiated:")
print(f"[INFO] -- Input channels (features × CWT scales): {NUM_CWT_CHANNELS}")
print(f"[INFO] -- Number of classes: {NUM_CLASSES}")
print(f"[INFO] -- Sequence length: {WINDOW_SIZE}")
print(model)

In [ ]:
# Defines loss function and optimizer
loss_criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

# Defines number of epochs for training
NUM_EPOCHS = 50
print(f"[INFO] -- Starting training for {NUM_EPOCHS} epochs...")

# Lists to store training loss and test accuracy for each epoch
train_losses_history = []
test_accuracies_history = []

# Learning rate scheduler
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.7)

# Training loop
for epoch in range(NUM_EPOCHS):
   # Sets the model to training mode
    model.train()
    total_train_loss = 0
    # Loops through training data in batches
    for batch_idx, (data_batch, labels_batch) in enumerate(train_loader):
        # Move batch data to the approriate device
        data_batch, labels_batch = data_batch.to(device), labels_batch.to(device)
        # Clears previous gradients
        optimizer.zero_grad()
        # Forward pass
        outputs_logits = model(data_batch)
        # Calculate loss
        loss = loss_criterion(outputs_logits, labels_batch)
        # Backpropagation
        loss.backward()
        # Updates model parameters
        optimizer.step()
        # Accumulated training loss
        total_train_loss += loss.item()
        # Prints training progress every 50 batches
        if (batch_idx + 1) % 50 == 0:
            print(f"Epoch [{epoch+1}/{NUM_EPOCHS}], Batch [{batch_idx+1}/{len(train_loader)}], Batch Loss: {loss.item():.4f}")
    # Computes average training loss for the epoch and store it
    avg_epoch_train_loss = total_train_loss / len(train_loader)
    train_losses_history.append(avg_epoch_train_loss)

    # Steps the learning rate scheduler
    scheduler.step()
    # Gets the current learning rate
    current_lr = scheduler.get_last_lr()[0]

    # Evaluation phase on test set
    # Sets model to evaluation mode
    model.eval()
    total_test_correct = 0
    total_test_samples = 0
    # Disables gradient calculation
    with torch.no_grad():
        for data_batch, labels_batch in test_loader:
            data_batch, labels_batch = data_batch.to(device), labels_batch.to(device)
            outputs_logits = model(data_batch)
            # Gets predicted class labels
            _, predicted_labels = torch.max(outputs_logits.data, 1)
            total_test_samples += labels_batch.size(0)
            total_test_correct += (predicted_labels == labels_batch).sum().item()
    # Calculates accuracy for the current epoch and stores it
    epoch_test_accuracy = 100 * total_test_correct / total_test_samples
    test_accuracies_history.append(epoch_test_accuracy)

    # Prints epoch summary
    print(f"Epoch [{epoch+1}/{NUM_EPOCHS}], Train Loss: {avg_epoch_train_loss:.4f}, Test Acc: {epoch_test_accuracy:.2f}%, LR: {current_lr:.6f}")
# Training complete
print("[INFO] -- Training Finished.")

In [ ]:
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(range(1, NUM_EPOCHS + 1), train_losses_history, label='Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss Over Epochs')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(range(1, NUM_EPOCHS + 1), test_accuracies_history, label='Test Accuracy', color='orange')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.title('Test Accuracy Over Epochs')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# Sets model to evaluation mode
model.eval()
# Lists to store all predicted labels and true labels
all_predicted_outputs = []
all_true_labels_eval = []

# Disable gradients computation for evaluation
with torch.no_grad():
    # Iterates through test set in batches
    for data_batch, labels_batch in test_loader:
        # Moves input data to the device (GPU or CPU)
        data_batch = data_batch.to(device)
        # Forward pass
        outputs_logits = model(data_batch)
         # Gets predicted class by taking the argmax of the output logits
        _, predicted_batch_labels = torch.max(outputs_logits.data, 1)
        # Stores predictions and corresponding true labels
        all_predicted_outputs.extend(predicted_batch_labels.cpu().numpy())
        all_true_labels_eval.extend(labels_batch.cpu().numpy()) # labels_batch are already on CPU from DataLoader

# Computes accuracy of final model on test set
final_model_accuracy = accuracy_score(all_true_labels_eval, all_predicted_outputs)
# Computes the confusion matrix
confusion_mat = confusion_matrix(all_true_labels_eval, all_predicted_outputs)
# Generates a detailed classification report including precision, recall, F1-score
classification_rep = classification_report(
    all_true_labels_eval, all_predicted_outputs, target_names=ACT_LABELS, zero_division=0
)
 # Prints evaluation results
print(f"\n[INFO] -- Final CWT-CNN Model Evaluation on Test Set:")
print(f"Overall Accuracy: {final_model_accuracy*100:.2f}%")
print("\nClassification Report:")
print(classification_rep)
print("\nConfusion Matrix:")

 # Plots the confusion matrix as a heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(confusion_mat, annot=True, fmt='d', cmap='Blues',
            xticklabels=ACT_LABELS, yticklabels=ACT_LABELS)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('CWT-CNN Confusion Matrix - Test Set')
plt.show()